In [43]:
import pandas as pd
from os import walk
import rdkit
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem import rdchem
from tqdm.notebook import tqdm
import seaborn
import matplotlib.pyplot as plt

from rdkit import RDLogger

lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

In [47]:
def pruner(dataframe):
    columns = ['WLN', 'SMILES']
    for i in dataframe:
        if i not in columns:
            dataframe = dataframe.drop(columns=i)
    return dataframe

def Carbon_Labeler(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
    except:
        label = 'Invalid-Mol'
        return label
    
    if mol:
        rings = rdkit.Chem.rdMolDescriptors.CalcNumRings(mol)        
        if rings >= 2:
            label = 'Multi-Ring'
        elif rings ==1:
            label = 'Single-Ring'
        else:
            carbon_check = []
            for atom in mol.GetAtoms():
                carbon = rdkit.Chem.rdchem.Atom.GetAtomicNum(atom)
                carbon_check.append(carbon)
                
            if 6 not in carbon_check:
                label = 'Non-Carbon'
                return label
            
            for atom in mol.GetAtoms():
                carbon = rdkit.Chem.rdchem.Atom.GetAtomicNum(atom)
                if carbon ==6:
                    x = rdkit.Chem.rdchem.Atom.GetDegree(atom)
                    if x >=3:
                        label = 'Branched'
                        break
                    elif x < 3:
                        label = 'Linear'
    else:
        label = 'Invalid-Mol'
    return label

def pruner(dataframe):
    columns = ['WLN', 'SMILES']
    for i in dataframe:
        if i not in columns:
            dataframe = dataframe.drop(columns=i)
    return dataframe

In [45]:
mypath= 'Generated_Data/'
filenames = next(walk(mypath), (None, None, []))[2]


data1 = pruner(pd.read_csv(mypath + filenames[0], sep='\t', names=['WLN', 'SMILES']))
data2 = pruner(pd.read_csv(mypath + filenames[1], sep='\t'))
data3 = pruner(pd.read_csv(mypath + filenames[2], sep='\t'))
data4 = pruner(pd.read_csv(mypath + filenames[4], sep='\t',names=['WLN', 'SMILES'] ))
data5 = pruner(pd.read_csv(mypath + filenames[5], sep='\t', names=['WLN', 'SMILES']))

data6 = pruner(pd.read_csv(mypath + filenames[6], sep='\t', names=['WLN', 'SMILES']))
data7 = pruner(pd.read_csv(mypath + filenames[7], sep='\t'))
data8 = pruner(pd.read_csv(mypath + filenames[8], sep='\t', names=['WLN', 'SMILES']))
data9 = pruner(pd.read_csv(mypath + filenames[9], sep='\t', names=['WLN', 'SMILES']))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 1082: invalid start byte

In [41]:
frames = [data1, data2, data3, data4, data5, data6, data7, data8, data9]
pre_merge = [pruner(i) for i in frames]
new_ring_data = pd.concat(pre_merge)

## Carbon Classing

In [49]:
Labels = [Carbon_Labeler(smiles) for smiles in new_ring_data['SMILES']]

In [50]:
new_ring_data['Labels'] = Labels
new_ring_data.head(10)

,WLN,SMILES,Labels
0,L6AV DVJ,O=C1C=CC(=O)C=C1,Single-Ring
1,L6AV DVJ BPFK,C[N+](C)(C)P(F)C1=CC(=O)C=CC1=O,Single-Ring
2,L6AV DVJ B6OM,[NH-]OCCCCCCC1=CC(=O)C=CC1=O,Single-Ring
3,L6AV DVJ B2K,C[N+](C)(C)CCC1=CC(=O)C=CC1=O,Single-Ring
4,L6AV DVJ BQ7S,O=C1C=CC(=O)C(OCCCCCCC[S-])=C1,Single-Ring
5,L6AV DVJ B37E,O=C1C=CC(=O)C(CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,Single-Ring
6,L6AV DVJ BQK4,CCCC[N+](C)(C)OC1=CC(=O)C=CC1=O,Single-Ring
7,L6AV DVJ BP91,CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,Single-Ring
8,L6AV DVJ B46E,O=C1C=CC(=O)C(CCCCCCCCCCCCCCCCCCCCCCCCCCCCCCCC...,Single-Ring
9,L6AV DVJ B3M2,CCNCCCC1=CC(=O)C=CC1=O,Single-Ring


In [51]:
processed = new_ring_data[new_ring_data.Labels != 'Invalid-mol']

In [42]:
processed.to_csv('STAGE2_GEN_ring_data.txt', sep='\t')